# Tarea: Algoritmo de Grover - David Salamanca

## Investigación

### El problema 3-SAT
El problema 3-SAT (abreviatura de **problema de 3-satisfacibilidad**) es un ejemplo clásico de un problema computacional clasificado como **NP-completo**, lo que resalta su complejidad y la dificultad para resolverlo bajo los paradigmas computacionales actuales. A continuación, se explica por qué este problema se considera difícil de resolver:

#### Definición de 3-SAT
3-SAT es un problema de satisfacibilidad booleano en el que se debe determinar si existe una asignación de valores de verdad (verdadero o falso) que haga verdadera una fórmula booleana. La fórmula se expresa en **forma normal conjuntiva (CNF)**, donde cada cláusula contiene exactamente tres literales (variables o sus negaciones).

#### NP-Completitud
El problema 3-SAT es **NP-completo**, lo que significa que:
1. Está en NP, ya que cualquier solución candidata puede verificarse en tiempo polinómico.
2. Es tan difícil como cualquier otro problema en NP, ya que cualquier problema NP puede reducirse a él en tiempo polinomial.

#### Crecimiento exponencial de posibles asignaciones
El número de posibles asignaciones de valores de verdad crece exponencialmente con el número de variables:
- Para un problema con `n` variables, hay \(2^n\) combinaciones posibles.
- Esto hace que los métodos de búsqueda exhaustiva sean imprácticos incluso para valores moderados de `n`.

#### Falta de algoritmos eficientes
A pesar de décadas de investigación, no se ha descubierto ningún algoritmo de tiempo polinomial para resolver problemas 3-SAT (o cualquier problema NP-completo) en computadoras clásicas.

#### Implicaciones del problema P vs NP
La insolubilidad del problema 3-SAT está vinculada al famoso problema **P vs NP**. Resolver 3-SAT en tiempo polinomial implicaría que **P = NP**, lo que revolucionaría la informática. Sin embargo, la mayoría de los expertos creen que **P ≠ NP**, lo que sugiere que los problemas NP-completos requieren tiempo superpolinomial para resolverse en el peor caso.

#### Impacto práctico
El problema 3-SAT tiene implicaciones en áreas como:
- **Criptografía**, donde la dificultad computacional protege datos y comunicaciones.
- **Diseño de algoritmos**, fomentando enfoques aproximados o probabilísticos para instancias grandes.

En resumen, el problema 3-SAT es un caso paradigmático de problemas que son fáciles de verificar, pero extremadamente difíciles de resolver.

---

## Taller

### Ejercicios de ejemplo 3-SAT
Para aplicar el algoritmo de Grover, se definen tres problemas de satisfacibilidad booleanos con tres variables cada uno. Cada problema utiliza cláusulas en forma normal conjuntiva (CNF).

#### Ejemplo 1
**Variables:** \(x\), \(y\), \(z\)  
**Cláusulas:**
1. \(x \lor \neg y \lor z\)
2. \(\neg x \lor y \lor \neg z\)
3. \(x \lor y \lor \neg z\)  

**Tarea:** Determinar los valores de verdad de \(x\), \(y\) y \(z\) que satisfacen todas las cláusulas.

#### Ejemplo 2
**Variables:** \(a\), \(b\), \(c\)  
**Cláusulas:**
1. \(a \lor \neg b \lor c\)
2. \(\neg a \lor \neg b \lor c\)
3. \(a \lor b \lor \neg c\)  

**Tarea:** Determinar los valores de verdad de \(a\), \(b\) y \(c\) que satisfacen todas las cláusulas.

#### Ejemplo 3
**Variables:** \(p\), \(q\), \(r\)  
**Cláusulas:**
1. \(\neg p \lor q \lor r\)
2. \(p \lor \neg q \lor r\)
3. \(p \lor q \lor \neg r\)  

**Tarea:** Determinar los valores de verdad de \(p\), \(q\) y \(r\) que satisfacen todas las cláusulas.

---

## Resolver con el Algoritmo de Grover

Para resolver los problemas de 3-SAT utilizando el algoritmo de Grover, siga estos pasos:

1. **Representación cuántica:**
   - Codificar cada variable y su negación como estados cuánticos.

2. **Aplicación del algoritmo de Grover:**
   - Utilizar el algoritmo para amplificar la probabilidad de los estados que satisfacen todas las cláusulas.

3. **Medición:**
   - Medir el registro cuántico para observar la solución.

4. **Informe:**
   - Documentar los experimentos y resultados obtenidos.

### Ventajas del algoritmo de Grover
El algoritmo de Grover es especialmente eficiente para buscar soluciones en grandes espacios de búsqueda. Para problemas de 3-SAT, puede encontrar soluciones válidas con un número cuadrático de iteraciones respecto al espacio de búsqueda, lo que representa una mejora significativa frente a los métodos clásicos.

In [ ]:
# Solución Tarea: Resolviendo un problema 3-SAT usando el Algoritmo de Grover

from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.visualization import plot_histogram

# Función para construir el Oracle que marca los estados válidos:
def oracle_3sat(qc, n_vars):
    """
    Implementa el Oracle para el problema 3-SAT.
    Fórmula: (x OR NOT y OR z) ∧ (NOT x OR y OR NOT z) ∧ (x OR y OR NOT z)

    Parámetros:
    - qc: QuantumCircuit, el circuito cuántico al que se agregará el Oracle.
    - n_vars: int, número de variables booleanas.
    """
    aux = n_vars  # Qubit auxiliar para los cálculos intermedios

    # Cláusula 1: (x OR NOT y OR z)
    qc.x(1)             # Aplicar NOT a y
    qc.ccx(0, 1, aux)   # AND entre x y NOT y
    qc.cx(2, aux)       # OR con z
    qc.x(1)             # Revertir NOT de y

    # Cláusula 2: (NOT x OR y OR NOT z)
    qc.x([0, 2])        # Aplicar NOT a x y z
    qc.ccx(0, 1, aux)   # AND entre NOT x y
    qc.cx(2, aux)       # OR con NOT z
    qc.x([0, 2])        # Revertir NOT de x y z

    # Cláusula 3: (x OR y OR NOT z)
    qc.x(2)             # Aplicar NOT a z
    qc.ccx(0, 1, aux)   # AND entre x y
    qc.cx(2, aux)       # OR con NOT z
    qc.x(2)             # Revertir NOT de z

    return qc

# Función para construir el operador de difusión:
def diffusion_operator(qc, n_vars):
    """
    Implementa el operador de difusión de Grover.

    Parámetros:
    - qc: QuantumCircuit, el circuito cuántico al que se agregará el operador.
    - n_vars: int, número de variables booleanas.
    """
    # Aplicar Hadamard y X gates
    qc.h(range(n_vars))
    qc.x(range(n_vars))
    
    # Aplicar una puerta multicontrolada (controlado-Z)
    qc.h(n_vars - 1)  # Puerta H al último qubit
    qc.mct(list(range(n_vars - 1)), n_vars - 1)  # Multicontrolada
    qc.h(n_vars - 1)  # Revertir la puerta H

    # Revertir X y Hadamard
    qc.x(range(n_vars))
    qc.h(range(n_vars))

    return qc

# Definir el número de variables y construir el circuito
n_vars = 3  # Número de variables booleanas
qc = QuantumCircuit(n_vars + 1, n_vars)  # Crear el circuito cuántico con un qubit auxiliar

# Inicialización: aplicar Hadamard para superposición cuántica
qc.h(range(n_vars))

# Agregar el Oracle y el operador de difusión
qc = oracle_3sat(qc, n_vars)
qc = diffusion_operator(qc, n_vars)

# Medir los qubits
qc.measure(range(n_vars), range(n_vars))

# Ejecutar el circuito en un simulador
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
result = execute(compiled_circuit, simulator, shots=1024).result()
counts = result.get_counts()

# Mostrar los resultados
print("Resultados:", counts)
plot_histogram(counts)